# Geocoding apartment addresses using the geoadmin API

## Libraries and settings

In [3]:
# Libraries
import os
import json
import urllib
import fnmatch
import math
import pandas as pd
from IPython.display import clear_output

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Importing diesel data

In [6]:
# Read the data to a pandas data frame
complete = pd.read_csv('..\Master\combined_loc.csv', 
                 sep=',', 
                 encoding='utf-8')

# Show first records
complete

,date,brand,address,diesel_price,bleifrei_price,lat,lon
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.120,1.140,46.943104,10.358743
1,28.10.2022,Socar Gotthard,"A2 West, 6467 Schattdorf",2.395,2.035,47.228649,7.621632
2,14.08.2022,BP Rheintal,"Autobahn A13, 9470 Buchs SG",2.380,2.200,NaN,NaN
3,11.07.2022,eni - Castasegna Ovest,"Farzett, 7608 Castasegna",2.380,2.230,47.285664,8.563127
4,03.08.2022,BP,"Grüningerstr. 80, 8624 Grüt (Gossau ZH)",2.370,2.220,47.448483,8.926337
...,...,...,...,...,...,...,...
2400,04.05.2016,Auto Bellinzona Nord Sagl,"Via S. Gottardo 57B, 6500 Bellinzona",1.320,1.330,NaN,NaN
2401,02.02.2016,Reisch AG,"Hauptstr. 24, 8573 Siegershausen",1.320,1.270,NaN,NaN
2402,04.05.2016,Auto Lugano,"Via Cantonale, 6983 Magliaso",1.319,1.279,NaN,NaN
2403,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.310,1.260,NaN,NaN


In [13]:
ort = []
for i in complete['address']:
    d1 = i.split(',')
    d2 = "".join(c for c in d1[1] if not c.isdecimal())
    ort.append(d2)

complete['ort'] = ort

complete

,date,brand,address,diesel_price,bleifrei_price,lat,lon,plz,ort
0,21.09.2020,Alpina Tankstelle,"Dorfstrasse 42, 7563 Samnaun Dorf",1.120,1.140,46.943104,10.358743,7563,Samnaun Dorf
1,28.10.2022,Socar Gotthard,"A2 West, 6467 Schattdorf",2.395,2.035,47.228649,7.621632,6467,Schattdorf
2,14.08.2022,BP Rheintal,"Autobahn A13, 9470 Buchs SG",2.380,2.200,NaN,NaN,9470,Buchs SG
3,11.07.2022,eni - Castasegna Ovest,"Farzett, 7608 Castasegna",2.380,2.230,47.285664,8.563127,7608,Castasegna
4,03.08.2022,BP,"Grüningerstr. 80, 8624 Grüt (Gossau ZH)",2.370,2.220,47.448483,8.926337,8624,Grüt (Gossau ZH)
...,...,...,...,...,...,...,...,...,...
2400,04.05.2016,Auto Bellinzona Nord Sagl,"Via S. Gottardo 57B, 6500 Bellinzona",1.320,1.330,NaN,NaN,6500,Bellinzona
2401,02.02.2016,Reisch AG,"Hauptstr. 24, 8573 Siegershausen",1.320,1.270,NaN,NaN,8573,Siegershausen
2402,04.05.2016,Auto Lugano,"Via Cantonale, 6983 Magliaso",1.319,1.279,NaN,NaN,6983,Magliaso
2403,22.12.2020,Ruedi Rüssel,"Luzernstr. 60, 4553 Subingen",1.310,1.260,NaN,NaN,4553,Subingen
